In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import diffusers
import transformers
import sys
import os
import shutil
import time
import subprocess
import torch
import matplotlib.pyplot as plt
import numpy as np
import diffusers
from PIL import Image

if torch.cuda.is_available():
    device_name = torch.device("cuda")
    torch_dtype = torch.float16
else:
    device_name = torch.device("cpu")
    torch_dtype = torch.float32

In [3]:
# pipe = diffusers.StableDiffusionPipeline.from_single_file("models/OhmwMan.ckpt")

# pipe.save_pretrained("OhmwMan")

In [4]:
# pipe = diffusers.StableDiffusionPipeline.from_single_file("realisticVisionV60B1_v51VAE.safetensors")

# pipe.save_pretrained("converted_realisticVisionV60B1_v51VAE")

model_path = "converted_realisticVisionV60B1_v51VAE"

In [ ]:

dreambooth_filename = "train_dreambooth.py"
img_folder_path = "images"
output_dir = "ohmwWithPPL"
class_prompt = "ohmw man"
IMG_SIZE = 512
TRAINING_STEPS = 2500
class_data_dir = "class_data_dir"


process = [
    "accelerate", 
    "launch",
    f"{dreambooth_filename}",
    "--train_text_encoder",
    "--allow_tf32",
    f"--pretrained_model_name_or_path={model_path}",
    f"--instance_data_dir={img_folder_path}",
    f"--output_dir={output_dir}",
    f"--instance_prompt='{class_prompt}'",
    f"--resolution={IMG_SIZE}",
    f"--train_batch_size=1",
    f"--gradient_accumulation_steps=1",
    "--gradient_checkpointing",
    f"--learning_rate=1e-6",
    f"--lr_scheduler=constant",
    f"--lr_warmup_steps=0",
    f"--max_train_steps={TRAINING_STEPS}",
    "--checkpointing_steps=500",
    "--mixed_precision=bf16",
]

# Extend process configuration
process.extend(
    [
        f"--with_prior_preservation",
        f"--prior_generation_precision=bf16",
        f"--sample_batch_size=1",
        f"--class_prompt='a man'",
        f"--class_data_dir={class_data_dir}",
    ]
)

process.extend(
    [
        f"--report_to=wandb",
        f'--validation_prompt="wide shot photo of ohmw man in a suit, professional, standing up, realistic, highly detailed"',
        f"--num_validation_images=3",
        f"--validation_steps=250",
        f"--seed=42",
    ]
)

# Run the process
subprocess.run(process, check=True)

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-02-20 03:49:11.362629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 03:49:11.362725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 03:49:11.540262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alread